<a href="https://colab.research.google.com/github/stgolarrain/tutorial-tensorflow2/blob/master/Totorial_Tensorflow_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial RecSys - Tensorflow

En este tutorial mostraremos cómo utilizar la librería Tensorflow para la implementación de un modelo de Factorización Matricial.

El modelo que implementaremos minimiza la siguiente función objetivo:

$\min_{p, q} \frac{1}{N} \sum_{(u,i) \in \mathcal{D}} (r_{ui} - p_u q_i)^2 + \lambda (||p_u||^2 + ||q_i||^2)$

Donde:
- $r_{ui}$ es el rating que le ha asignado el usuario $u$ al ítem $i$ en el set de entrenamiento.
- $p_u$ representa el vector latente del usuario.
- $q_i$ representa el vector latente del ítem.
- $\lambda$ es la variable regularizadora.


## Configuración del entorno

En esta sección descargaremos el set de datos que utilizaremos para trabajar e instalaremos las dependencias necesarias para implementar el modelo.

In [0]:
# Creamos una carpeta donde guardaremos los datos
!mkdir data

# Descargamos y descomprimimos los datos
!curl -L -o "data/ml-latest-small.zip" "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
!unzip data/ml-latest-small.zip -d data

# Vemos las primeras líneas del archivo para asegurar que la descarga haya sido correcta
!head data/ml-latest-small/ratings.csv

Cada línea representa un usuario, ítem del producto, rating y timestamp de la hora que se hizo la evaluación del ítem. Por ejemplo:
```
1;1193;5;978300760
```
Significa que el usuario 1 le dió rating 5 al ítem 1193 en el tiempo 978300760. En este [link](https://https://www.timestampconvert.com/?go2=true&offset=4&timestamp=978300760&Submit=++++++Convert+to+Date++++++) podemos ver la transformación de 978300760 a una fecha normal que sería: 31/12/2000 19:12:40 de Chile.

Instalamos las dependencias necesarias.

In [0]:
!pip install tensorflow --upgrade

## Carga y preprocesamiento de datos

Los datos a utilizar provienen de un dataset bastante popular ([MovieLens](https://grouplens.org/datasets/movielens/)) y que ha sido utilizado en otros prácticos de este curso. A continuación, se importarán los datos utilizando pandas para facilitar su manejo y obtener algunos valores interesantes de los datos:

In [0]:
# Pandas facilita el manejo de datos como tablas
import pandas as pd

# Cargar datos
data_path = 'data/ml-latest-small/ratings.csv'
data = pd.read_csv(data_path)

# Imprimir tamaños relevantes
print("Numero de usuarios: {}".format(data['userId'].nunique()))
print("Numero de items: {}".format(data['movieId'].nunique()))
print("Numero de datos (filas): {}".format(data.shape[0]))
print()

# Vemos las primero 5 filas de los datos
data.head(5)

Ahora utilizaremos diccionarios para transformar los identificadores de usuario y películas a nuevos identificadores, que van desde 0 hasta N (la cantidad de datos de cada entidad):

In [0]:
# A cada ID del dataset, le asigno un nuevo ID de 0 a N
user_to_int = {user: i for i, user in enumerate(data['userId'].unique())}
item_to_int = {item: i for i, item in enumerate(data['movieId'].unique())}

# Aplicamos la transformacion de ID ya creada
data['userId'] = data['userId'].map(lambda user: user_to_int[user])
data['movieId'] = data['movieId'].map(lambda item: item_to_int[item])

data.head(5)

El total de los datos los separaremos en un set de entrenamiento (80%) y en un set de validación (20%).

In [0]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data, test_size=0.2)

print(f'Datos de entrenamiento: {train_data.shape[0]}')
print(f'Datos de validación: {val_data.shape[0]}')

Por último ambos set de datos los transformaremos en un objeto [DataSet](https://www.tensorflow.org/api_docs/python/tf/data/Dataset), lo que genera un _stream_ de datos configurable para conseguir los datos en forma de _batch_.

In [0]:
import tensorflow as tf

def create_dataset(df, batch_size=128):
  x = df[['userId', 'movieId']].values
  y = df[['rating']].values
  
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.batch(batch_size)
  return ds

train_ds = create_dataset(train_data)
val_ds = create_dataset(val_data)

El objeto DataSet puede ser usado como un iterador donde cada iteración devuelve los datos de entrenamiento en formato _batch_: Los input $x$ y outptu $y$. En este caso los datos $x$ son pares (`userId`, `movieId`) y los datos $y$ corresponden a los datos de `rating`. Cada _batch_ es de largo 32 para este ejemplo.

In [0]:
# Ejemplo de cómo iterar sobre el dataset
for (x, y) in train_ds:
  print(x.shape)
  print(y.shape)
  break

## Factorización matricial

En esta sección implementaremos el modelo con la API de tensorflow y luego entrenaremos el modelo con los datos descargados.

La implementación del modelo se compone en dos partes:
1.   En la primera parte implementaremos la capa interna, en la cual implementaremos una clase `Layer` de la API de Keras. Se recomienda ver la [documentación](https://www.tensorflow.org/guide/keras/custom_layers_and_models#the_layer_class) de dicha clase. Las clases `Layer` encapsulan los "pesos" de los modelos y su cómputo interno. Para efectos de la factorización matricial, los pesos son las matrices $P$ y $Q$.
2.   En la segunda parte implementaremos una clase `Model` de la API de Keras. Al igual que la clase `Layer` se recomienda revisar la [documentación](https://www.tensorflow.org/guide/keras/custom_layers_and_models#building_models) para más detalles. Las clases `Model` encapsulan la capa externa del modelo que queremos implementar y definen métodos ya implementados que son de alta utilida (`model.fit()`, `model.evaluate()`, `model.predict()`).



Ahora continuaremos implementando la capa interna del modelo. La capa interna crea las variables ($P$ y $Q$) que serán modificadas en el proceso de entrenamiento. La clase implementa 3 métodos principales:

*   `__init__`: inicializa la clase y asigna internamente los parámetros del modelo, i.e. los tamaños de las matrices $P$ y $Q$.
*   `build`: construye las variables necesarias para realizar los cálculos internos.
*   `call`: recibe como parámetros los índice de los usuarios e items, busca los _embeddings_ y realiza la multiplicación de los mismos.


In [0]:
# Implementación de la clase `Layer`
class FactorizationLayer(tf.keras.layers.Layer):
  def __init__(self, n_users, n_items, k=32):
    super().__init__()
    self.k = k
    self.n_users = n_users
    self.n_items = n_items

  def build(self, input_shape):
    # Creamos la matriz P con los vectores latentes de los usuarios
    self.P = self.add_variable('P_matrix',
        shape=[self.n_users, self.k],
        dtype=tf.float32,
        initializer=tf.random_uniform_initializer(minval=-1., maxval=1.))

    # Creamos la matriz Q con los vectores latentes de los items
    self.Q = self.add_variable('Q_matrix',
        shape=[self.n_items, self.k],
        dtype=tf.float32,
        initializer=tf.random_uniform_initializer(minval=-1., maxval=1.))

  def call(self, inputs):
    # Separamos los indice de usuarios e items
    user_input = inputs[:, 0]
    item_input = inputs[:, 1]

    # Buscamos el vector latente de usuario e item con la
    # función `embedding_lookup`
    p = tf.nn.embedding_lookup(self.P, user_input, name='user_embedding_lookup')
    q = tf.nn.embedding_lookup(self.Q, item_input, name='item_embedding_lookup')

    # Multiplicamos los embedding de usuario e items
    y_hat = tf.reduce_sum(tf.multiply(p, q), 1)

    return y_hat, p, q

Ahora continuaremos implementando la capa externa del modelo heredando de la clase `tf.keras.Model`. Esta clase implementa los siguientes métodos:

*   `__init__`: método que inicializa el modelo.
*   `call`: implementa las operaciones del modelo y las capas internas.

Recordemos que además de los métodos implementados, al heredar de la clase `Model` también tenemos los métdos `fit()`, `evaluate()`, `predict()`.



In [0]:
class MatrixFactorization(tf.keras.Model):
  def __init__(self, n_users, n_items, k=32, alpha=.01):
    super().__init__()
    self.alpha = alpha
    self.layer = FactorizationLayer(n_users, n_items, k=k)

  def call(self, inputs):
    # Las predicciones del modelo
    y_hat, p, q = self.layer(inputs)

    # Además calculamos el regularizador
    l2_loss = self.alpha * (tf.nn.l2_loss(p) + tf.nn.l2_loss(q))
    self.add_loss(l2_loss)
    return y_hat

Ahora instanciaremos y entrenaremos el modelo. Utilizando los métodos de la clase `Model`.

In [0]:
n_users = data['userId'].nunique()
n_items = data['movieId'].nunique()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss = tf.keras.losses.MeanSquaredError()

model = MatrixFactorization(n_users, n_items)
model.compile(optimizer=optimizer, loss=mse_loss)
model.fit(train_ds, 
          epochs=20, 
          verbose=1,
          validation_data=val_ds)

## Tensorboard

**Tensorboard** nos permite visualizar algunas variables del modelo y el grafo de cómputo generado.

In [0]:
import datetime, os

logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
writer = tf.summary.create_file_writer(logdir)

model = MatrixFactorization(n_users, n_items)
model.compile(optimizer=optimizer, loss=mse_loss)

model.fit(train_ds, 
          epochs=10, 
          verbose=1,
          validation_data=val_ds,
          callbacks=[tensorboard_callback])

tf.summary.trace_on(graph=True, profiler=True)
with writer.as_default():
  tf.summary.trace_export(
      name='Tutorial_Recsys',
      step=0,
      profiler_outdir=logdir)

In [0]:
# Cargamos la extensión de tensorboard y lo ejecutamos
%load_ext tensorboard
%tensorboard --logdir logs